## SciBERT is a BERT model trained on scientific text.

* SciBERT is trained on papers from the corpus of semanticscholar.org. Corpus size is 1.14M papers, 3.1B tokens 
 
* It is trained on the full text of the papers in training,and not just abstracts

* It has its own vocabulary (scivocab) that's built to best match the training corpus

The following is an implementation of SciBert via TensorFlow2

In [1]:
!pip install -q transformers==2.11.0
!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/tensorflow_models/scibert_scivocab_uncased.tar.gz
!tar -xvf ./scibert_scivocab_uncased.tar.gz

ERROR: azureml-automl-dnn-nlp 1.40.0 has requirement transformers<=4.5.1,>=4.1.0, but you'll have transformers 2.11.0 which is incompatible.
--2022-09-15 07:48:43--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/tensorflow_models/scibert_scivocab_uncased.tar.gz
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.92.163.32
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.92.163.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1216161420 (1.1G) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar.gz’

scibert_scivocab_un 100%[===================>]   1.13G  36.7MB/s    in 34s     

2022-09-15 07:49:22 (34.6 MB/s) - ‘scibert_scivocab_uncased.tar.gz’ saved [1216161420/1216161420]

scibert_scivocab_uncased/
scibert_scivocab_uncased/bert_model.ckpt.data-00000-of-00001
scibert_scivocab_uncased/bert_model.ckpt.index
scibert_scivocab_uncased/vocab.txt
scibert_scivocab_uncased/bert_model.ckpt.meta
scibert

In [2]:
os.environ["WANDB_API_KEY"] = "0" ## to silence warning
!transformers-cli convert --model_type bert \
  --tf_checkpoint './scibert_scivocab_uncased/bert_model.ckpt' \
  --config './scibert_scivocab_uncased/bert_config.json' \
  --pytorch_dump_output './scibert_scivocab_uncased/pytorch_model.bin'

Building PyTorch model from configuration: BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 31090
}

INFO:transformers.modeling_bert:Converting TensorFlow checkpoint from /mnt/batch/tasks/shared/LS_root/mounts/clusters/dsvdm/code/Users/Tamilselvan.S/Press Release/Code/scibert_scivocab_uncased/bert_model.ckpt
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/LayerNorm/beta/adam_m with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/LayerNorm/beta/adam_v with shape [768]
INFO:transformers.modeling_bert:Loading TF 

In [1]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import tensorflow as tf

from transformers import BertTokenizer ,BertConfig
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from transformers import TFBertModel
from tensorflow.keras.layers import Dense, Flatten
# from transformers import BertModel,BertForMaskedLM
import time
from transformers import create_optimizer
import transformers

print('transformers version :', transformers.__version__)

transformers version : 2.11.0


# The model has been trained on 4 modifications of data:

* Original
* Original without Latex Tags
* Augmented Data(Translate + De-Translate)
* Augmented Data - without Latex Tags


In [2]:
df = pd.read_csv("/mnt/batch/tasks/shared/LS_root/mounts/clusters/dsvdm/code/Users/Tamilselvan.S/Press Release/Data/PR_FINAL_CLEANED_SCRAPPED.csv")
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2,random_state=22)
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
train_data.head()

Category                                           URL_TEXT  \
0            Won Project  New research shows 77% of organizations are no...   
1  Corporate Information    NTT DATA Rises to No. 6 in Gartner Market Sh...   
2   Merger & Acquisition  New research shows 77% of organizations are no...   
3   Award/Contest Winner    NTT DATA Named a Leader in Everest Group's H...   
4                 Others  New research shows 77% of organizations are no...   

                                          URL_TEXT_1  \
0  77 organizations prepared change digital trans...   
1  rises 6 gartner market share services worldwid...   
2  77 organizations prepared change digital trans...   
3  named leader everest groups healthcare payer d...   
4  77 organizations prepared change digital trans...   

                                                 url  \
0  https://us.nttdata.com/en/news/press-release/2...   
1  https://www.nttdata.com/global/en/media/press-...   
2  https://us.nttdata.com/en/news/press-release/2...   
3  https://www.nttdata.com/global/en/media/press-...   
4  https://us.nttdata.com/en/news/press-release/2...   

                                          URL_TEXT_2  \
0  77 organizations prepared change digital trans...   
1  rises 6 gartner market share services worldwid...   
2  77 organizations prepared change digital trans...   
3  named leader everest groups healthcare payer d...   
4  77 organizations prepared change digital trans...   

                                               FINAL  
0  organizations technology group group indycar t...  
1  rises services 19 corporation 19 global servic...  
2  organizations technology group group indycar g...  
3  services corporation services group group serv...  
4  organizations technology group group indycar q...

In [10]:
# train_data['Category'].value_counts()]

In [4]:
train_data[train_data['FINAL'].isnull().values]['Category'].value_counts()

Series([], Name: Category, dtype: int64)

In [5]:
test_data[test_data['FINAL'].isnull().values]['Category'].value_counts()

Series([], Name: Category, dtype: int64)

In [3]:
train_data.columns,test_data.columns

(Index(['Category', 'URL_TEXT', 'URL_TEXT_1', 'url', 'URL_TEXT_2', 'FINAL'], dtype='object'),
 Index(['Category', 'URL_TEXT', 'URL_TEXT_1', 'url', 'URL_TEXT_2', 'FINAL'], dtype='object'))

In [6]:
# train_data['text'] = train_data['Title / Contents1'] + " " + train_data['Description1']
# test_data['text'] = test_data['Title / Contents1'] + " " + test_data['Description1']
train_data['text'] = train_data['FINAL']
test_data['text'] = test_data['FINAL']


In [7]:
# import string
# PUNCT_TO_REMOVE = string.punctuation

# def remove_punctuation(text):

#     """custom function to remove the punctuation"""

#     return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))



# train_data["sentence"] = train_data["sentence"].apply(lambda text: remove_punctuation(text))
# test_data["sentence"] = test_data["sentence"].apply(lambda text: remove_punctuation(text))

# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# ", ".join(stopwords.words('english'))
# STOPWORDS = set(stopwords.words('english'))

In [8]:
# import re
# def remove_stopwords(text):
#     """custom function to remove the stopwords"""
#     return " ".join([word for word in str(text).split() if word not in STOPWORDS])



# train_data["sentence"] = train_data["sentence"].apply(lambda text: remove_stopwords(text))
# test_data["sentence"] = test_data["sentence"].apply(lambda text: remove_stopwords(text))

# def clean_text(text):
#     '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
#     and remove words containing numbers.'''
#     text = str(text).lower()
#     text = re.sub('\[.*?\]', '', text)
#     text = re.sub('https?://\S+|www\.\S+', '', text)
#     text = re.sub('<.*?>+', '', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#     text = re.sub('\n', '', text)
#     text = re.sub('\w*\d\w*', '', text)
#     return text

# train_data["sentence"] = train_data["sentence"].apply(lambda text: clean_text(text))
# test_data["sentence"] = test_data["sentence"].apply(lambda text: clean_text(text))

In [9]:
# def remove_newlines_tabs(text):
#     """    
#     Example:
#     Input : This is her \\ first day at this place.\n Please,\t Be nice to her.\\n
#     Output : This is her first day at this place. Please, Be nice to her. 
    
#     """
    
#     Formatted_text = str(text).replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
#     return Formatted_text

# train_data["sentence"] = train_data["sentence"].apply(lambda text: remove_newlines_tabs(text))
# test_data["sentence"] = test_data["sentence"].apply(lambda text: remove_newlines_tabs(text))

In [10]:
# def remove_whitespace(text):
#     """ 
#     Example:
#     Input : How   are   you   doing   ?
#     Output : How are you doing ?     
        
#     """
#     pattern = re.compile(r'\s+') 
#     Without_whitespace = re.sub(pattern, ' ', text)
#     text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
#     return text

# train_data["sentence"] = train_data["sentence"].apply(lambda text: remove_whitespace(text))
# test_data["sentence"] = test_data["sentence"].apply(lambda text: remove_whitespace(text))

In [11]:
# import unidecode
# def accented_characters_removal(text):
#     # this is a docstring
#     """
        
#     Example:
#     Input : Málaga, àéêöhello
#     Output : Malaga, aeeohello    
        
#     """
#     text = unidecode.unidecode(text)
#     return text

# train_data["sentence"] = train_data["sentence"].apply(lambda text: accented_characters_removal(text))
# test_data["sentence"] = test_data["sentence"].apply(lambda text: accented_characters_removal(text))

In [12]:
# def removing_special_characters(text):
#     """Removing all the special characters except the one that is passed within 
#        the regex to match, as they have imp meaning in the text provided.
   
    
#     arguments:
#          input_text: "text" of type "String".
         
#     return:
#         value: Text with removed special characters that don't require.
        
#     Example: 
#     Input : Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) 
#     Output :  Hello, Kajal. This is $100.05 : the payment that you will recieve! Is this okay?
    
#    """
#     Formatted_Text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text)
#     return Formatted_Text

# train_data["sentence"] = train_data["sentence"].apply(lambda text: removing_special_characters(text))
# test_data["sentence"] = test_data["sentence"].apply(lambda text: removing_special_characters(text))

In [13]:
# def remove_punctuation(text):

#     """custom function to remove the punctuation"""
#     i = str(text).replace("(r)",'')
#     return i.translate(str.maketrans('', '', PUNCT_TO_REMOVE)).strip()


# train_data["sentence"] = train_data["sentence"].apply(lambda text: remove_punctuation(text))
# test_data["sentence"] = test_data["sentence"].apply(lambda text: remove_punctuation(text))

In [14]:
train_data["text"].sample(3).values

array(['fact 100percent 100percent companies gradually clark chief global services services newark califbased clark chief services fact clark companies gradually ensono acquires amido services crns clients clark ohio surrounded clark noncloud time clark observers companies financial string group analytics acorio clark kovar nontechfocused beats services reached',
       'announced partnership global services olives deploy develop partnership olives open olives codevelop olives follow tmobile canva costsavings roku roku group services companies chief industry services services olives ai trzmo 12 ait group reporters technology ai itops cyberthreat analytics ait newsmartechseriescom secureworks handsonkeyboard detector apatics tmobile canva costsavings roku roku vizio scripps ai organizations ai technology globe tech ai tech events ©',
       'technology 27 corporation 27 global services technology compilation technology technology composed sections tech tech dive technology futureoriente

In [15]:
test_data["text"].sample(3).values

array(['organizations technology group group indycar services announced aaron millstone services millstone services millstone oracles services 23 ranged global offering millstone companies acorio clients aaron joins time aarons global aaron global clients group services services millstone clients global master johns hopkins services announced aaron millstone services millstone services millstone oracles services 23 ranged global offering millstone companies acorio clients aaron joins time aarons global aaron global clients group services services millstone clients global master johns hopkins group global services clients industry services clients clients success combine global',
       'organizations technology group group indycar technology global services announced ai ai entries geek squad nlp ai symptoms preliminary firstline dogscats polycystic ovary syndrome pcos epileptic seizures global services people entries 22 jury inr 150000 inr 125000 inr 75000 technology global services an

In [16]:
train_data.shape,test_data.shape

((256, 7), (65, 7))

In [17]:
train_data['Category'].value_counts(normalize=True)*100

Corporate Information         31.640625
Offering                      17.968750
AR Report / Market Insight    16.406250
Others                        12.109375
Business Partnership          10.156250
Merger & Acquisition           4.296875
Award/Contest Winner           3.906250
Won Project                    3.515625
Name: Category, dtype: float64

In [18]:
train_data['Category'].value_counts().index

Index(['Corporate Information', 'Offering', 'AR Report / Market Insight',
       'Others', 'Business Partnership', 'Merger & Acquisition',
       'Award/Contest Winner', 'Won Project'],
      dtype='object')

In [19]:
train_data.head()

Category                                           URL_TEXT  \
0            Won Project  New research shows 77% of organizations are no...   
1  Corporate Information    NTT DATA Rises to No. 6 in Gartner Market Sh...   
2   Merger & Acquisition  New research shows 77% of organizations are no...   
3   Award/Contest Winner    NTT DATA Named a Leader in Everest Group's H...   
4                 Others  New research shows 77% of organizations are no...   

                                          URL_TEXT_1  \
0  77 organizations prepared change digital trans...   
1  rises 6 gartner market share services worldwid...   
2  77 organizations prepared change digital trans...   
3  named leader everest groups healthcare payer d...   
4  77 organizations prepared change digital trans...   

                                                 url  \
0  https://us.nttdata.com/en/news/press-release/2...   
1  https://www.nttdata.com/global/en/media/press-...   
2  https://us.nttdata.com/en/news/press-release/2...   
3  https://www.nttdata.com/global/en/media/press-...   
4  https://us.nttdata.com/en/news/press-release/2...   

                                          URL_TEXT_2  \
0  77 organizations prepared change digital trans...   
1  rises 6 gartner market share services worldwid...   
2  77 organizations prepared change digital trans...   
3  named leader everest groups healthcare payer d...   
4  77 organizations prepared change digital trans...   

                                               FINAL  \
0  organizations technology group group indycar t...   
1  rises services 19 corporation 19 global servic...   
2  organizations technology group group indycar g...   
3  services corporation services group group serv...   
4  organizations technology group group indycar q...   

                                                text  
0  organizations technology group group indycar t...  
1  rises services 19 corporation 19 global servic...  
2  organizations technology group group indycar g...  
3  services corporation services group group serv...  
4  organizations technology group group indycar q...

In [20]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
oc = enc.fit_transform(train_data['Category'].values.reshape(-1, 1))
y = train_data["Category"]
del train_data['Category']

train_data = pd.concat([train_data,pd.DataFrame(oc.toarray(),columns=sorted(list(y.unique()),reverse=False))],axis=1)

In [21]:
for col in list(train_data.columns[-8:]):
    train_data[col] = train_data[col].astype(int)
    
train_data.head()

URL_TEXT  \
0  New research shows 77% of organizations are no...   
1    NTT DATA Rises to No. 6 in Gartner Market Sh...   
2  New research shows 77% of organizations are no...   
3    NTT DATA Named a Leader in Everest Group's H...   
4  New research shows 77% of organizations are no...   

                                          URL_TEXT_1  \
0  77 organizations prepared change digital trans...   
1  rises 6 gartner market share services worldwid...   
2  77 organizations prepared change digital trans...   
3  named leader everest groups healthcare payer d...   
4  77 organizations prepared change digital trans...   

                                                 url  \
0  https://us.nttdata.com/en/news/press-release/2...   
1  https://www.nttdata.com/global/en/media/press-...   
2  https://us.nttdata.com/en/news/press-release/2...   
3  https://www.nttdata.com/global/en/media/press-...   
4  https://us.nttdata.com/en/news/press-release/2...   

                                          URL_TEXT_2  \
0  77 organizations prepared change digital trans...   
1  rises 6 gartner market share services worldwid...   
2  77 organizations prepared change digital trans...   
3  named leader everest groups healthcare payer d...   
4  77 organizations prepared change digital trans...   

                                               FINAL  \
0  organizations technology group group indycar t...   
1  rises services 19 corporation 19 global servic...   
2  organizations technology group group indycar g...   
3  services corporation services group group serv...   
4  organizations technology group group indycar q...   

                                                text  \
0  organizations technology group group indycar t...   
1  rises services 19 corporation 19 global servic...   
2  organizations technology group group indycar g...   
3  services corporation services group group serv...   
4  organizations technology group group indycar q...   

   AR Report / Market Insight  Award/Contest Winner  Business Partnership  \
0                           0                     0                     0   
1                           0                     0                     0   
2                           0                     0                     0   
3                           0                     1                     0   
4                           0                     0                     0   

   Corporate Information  Merger & Acquisition  Offering  Others  Won Project  
0                      0                     0         0       0            1  
1                      1                     0         0       0            0  
2                      0                     1         0       0            0  
3                      0                     0         0       0            0  
4                      0                     0         0       1            0

In [22]:
label_cols = sorted(list(y.unique()),reverse=False)
label_cols

['AR Report / Market Insight',
 'Award/Contest Winner',
 'Business Partnership',
 'Corporate Information',
 'Merger & Acquisition',
 'Offering',
 'Others',
 'Won Project']

In [23]:
# Parameters

BATCH_SIZE = 8

TEST_BATCH_SIZE = 8

NR_EPOCHS = 5

MAX_LEN = 256 # try diffrent lengths

threshold = 0.5

bert_model_name = './scibert_scivocab_uncased'

config = BertConfig.from_json_file('./scibert_scivocab_uncased/bert_config.json')

# Tokenizer

In [24]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)

def tokenize_sentences(sentences, tokenizer, max_seq_len = 256):
    
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        
        tokenized_sentence = tokenizer.encode(sentence,                  
                                              add_special_tokens = True, 
                                              max_length = max_seq_len  
                                             )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        
        att_mask = [int(token_id > 0) for token_id in sentence]
        
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

input_ids = tokenize_sentences(train_data['text'], tokenizer, MAX_LEN)

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

attention_masks = create_attention_masks(input_ids)

  0%|          | 0/256 [00:00<?, ?it/s]

# SciBERT Classifier

In [25]:
class BertClassifier(tf.keras.Model):    
    
    def __init__(self, bert: TFBertModel, num_classes: int):
        
        super().__init__()
        
        self.bert = bert
        
        self.classifier = Dense(num_classes, activation='sigmoid')
        
    def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)
        
        cls_output = outputs[1]
        
        cls_output = self.classifier(cls_output)
                
        return cls_output

In [26]:
def create_dataset(data_tuple, batch_size, train=True):
    
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    
    dataset = dataset.repeat(1)
    
    if train:
        
        dataset = dataset.shuffle(buffer_size=10000)
    
    dataset = dataset.batch(batch_size)
    
    if train:
        
        dataset = dataset.prefetch(1)
        
    if not train:
        
        dataset = dataset.cache()
    
    return dataset


In [27]:
def create_test_dataset():
    
    test_input_ids = tokenize_sentences(test_data['text'], tokenizer, MAX_LEN)
    
    test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
    
    test_attention_masks = create_attention_masks(test_input_ids)
    
    test_steps = len(test_data) // TEST_BATCH_SIZE
    
    test_dataset = create_dataset((test_input_ids, test_attention_masks), batch_size=TEST_BATCH_SIZE, train=False)
    
    return test_dataset

test_data['ID'] = [i for i in range(test_data.shape[0])]
submission = pd.DataFrame(columns=['ID','label','text','url'])
submission['ID'] = [i for i in range(test_data.shape[0])]
submission['label'] = test_data['Category']
submission['text'] = test_data['text']
submission['url'] = test_data['url']

enc = OneHotEncoder()
ocs = enc.fit_transform(submission['label'].values.reshape(-1, 1))
ys = submission["label"]
del submission['label']
submission = pd.concat([submission,pd.DataFrame(ocs.toarray(),columns=sorted(list(ys.unique()),reverse=False))],axis=1)
submission.set_index('ID',inplace=True)

for col in sorted(list(ys.unique()),reverse=False):
    submission[col] = submission[col].astype(int)

def generate_class_probablities(model,test_dataset,test_steps):
    
    for i, (token_ids, masks) in enumerate(tqdm(test_dataset, total=test_steps)):
        
        sample_ids = test_data.iloc[i*TEST_BATCH_SIZE:(i+1)*TEST_BATCH_SIZE]['ID']
        
        predictions = model(token_ids, attention_mask=masks).numpy()
        
        submission.loc[sample_ids, label_cols] = predictions
    
    return submission.loc[:,label_cols].values

test_dataset = create_test_dataset()

test_steps = len(test_data) // TEST_BATCH_SIZE

class_probs = np.zeros(shape =(len(test_data),len(label_cols)))

  0%|          | 0/65 [00:00<?, ?it/s]

In [28]:
#Gradient Calculation

def train_step(model, token_ids, masks, labels):
    
    labels = tf.dtypes.cast(labels, tf.float32)
    
    with tf.GradientTape() as tape:
        
        predictions = model(token_ids, attention_mask=masks)
        
        loss = loss_object(labels, predictions)
    
    
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables), name = 'gradients')
    
    train_loss(loss)

    for i, auc in enumerate(train_auc_metrics):
        
        auc.update_state(labels[:,i], predictions[:,i])
        
def validation_step(model, token_ids, masks, labels):
    
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    
    v_loss = loss_object(labels, predictions)

    validation_loss(v_loss)
    
    for i, auc in enumerate(validation_auc_metrics):
        
        auc.update_state(labels[:,i], predictions[:,i])                                    

# Model

In [29]:
seeds = [0]

for seed in range(len(seeds)):
    
    print('=' * 50, f"CV {seed+1}", '=' * 50)
    
    model = BertClassifier(TFBertModel.from_pretrained(bert_model_name, from_pt=True, config = config), len(label_cols))
    
    labels =  train_data[label_cols].values
    
    train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=seed, test_size = 0.2)

    train_masks, validation_masks = train_test_split(attention_masks, random_state=seed, test_size=0.2)

    train_size = len(train_inputs)

    validation_size = len(validation_inputs)


    train_dataset = create_dataset((train_inputs, train_masks, train_labels), batch_size=BATCH_SIZE,train=True)

    validation_dataset = create_dataset((validation_inputs, validation_masks, validation_labels), batch_size=BATCH_SIZE,train=False)
    
    
    steps_per_epoch = train_size // (BATCH_SIZE)

    #  Loss Function
    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    train_loss = tf.keras.metrics.Mean(name='train_loss')

    validation_loss = tf.keras.metrics.Mean(name='val_loss')

    #  Optimizer (with 1-cycle-policy)
    warmup_steps = steps_per_epoch // 3

    total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps

    optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

    # Gradients
    
    gradients = 0
    
    #  Metrics
    train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
#     train_f1_metrics = [f1_score for i in range(len(label_cols))]

    validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]


    for epoch in range(NR_EPOCHS):

        print('=' * 50, f"EPOCH {epoch+1}", '=' * 50)

        start = time.time()


        for batch_no, (token_ids, masks, labels) in enumerate(tqdm(train_dataset)):

            train_step(model, token_ids, masks, labels)

            if batch_no % 100 == 0:

                    print(f'\nTrain Step: {batch_no}, Loss: {train_loss.result()}')

                    for i, label_name in enumerate(label_cols):

                        print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")

                        train_auc_metrics[i].reset_states()

        for batch_no, (token_ids, masks, labels) in enumerate(tqdm(validation_dataset)):

            validation_step(model, token_ids, masks, labels)

        print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_cols):

            print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")

            validation_auc_metrics[i].reset_states()

        print('\n')
        
    probs = generate_class_probablities(model,test_dataset,test_steps)
        
    submission.loc[:, label_cols] = probs
        
    submission.to_csv('probs'+str(seed)+'.csv')
        
    class_probs += probs

================================================== CV 1 ==================================================
================================================== EPOCH 1 ==================================================


0it [00:00, ?it/s]


Train Step: 0, Loss: 0.7077158689498901
AR Report / Market Insight roc_auc 0.1428571343421936
Award/Contest Winner roc_auc 1.0
Business Partnership roc_auc 0.0
Corporate Information roc_auc 0.875
Merger & Acquisition roc_auc 0.0
Offering roc_auc 0.3333333134651184
Others roc_auc 0.0
Won Project roc_auc 0.5833333730697632


0it [00:00, ?it/s]


Epoch 1, Validation Loss: 0.33454611897468567, Time: 136.15130019187927

AR Report / Market Insight roc_auc 0.37139689922332764
Award/Contest Winner roc_auc 0.38235294818878174
Business Partnership roc_auc 0.6822916269302368
Corporate Information roc_auc 0.7189849615097046
Merger & Acquisition roc_auc 0.0
Offering roc_auc 0.529933512210846
Others roc_auc 0.640625
Won Project roc_auc 0.27551019191741943


================================================== EPOCH 2 ==================================================


...

0it [00:00, ?it/s]


Train Step: 0, Loss: 0.33191919326782227
AR Report / Market Insight roc_auc 0.9157088398933411
Award/Contest Winner roc_auc 0.8880341649055481
Business Partnership roc_auc 0.9395380020141602
Corporate Information roc_auc 0.8600063323974609
Merger & Acquisition roc_auc 0.9406499862670898
Offering roc_auc 0.799830973148346
Others roc_auc 0.9244080185890198
Won Project roc_auc 0.8379629850387573


0it [00:00, ?it/s]


Epoch 5, Validation Loss: 0.335995078086853, Time: 132.1387803554535

AR Report / Market Insight roc_auc 0.6219512224197388
Award/Contest Winner roc_auc 0.21568626165390015
Business Partnership roc_auc 0.7942708134651184
Corporate Information roc_auc 0.7894736528396606
Merger & Acquisition roc_auc 0.0
Offering roc_auc 0.5454545617103577
Others roc_auc 0.5696023106575012
Won Project roc_auc 0.5680272579193115




  0%|          | 0/8 [00:00<?, ?it/s]

# Submission

In [30]:
sub = submission.copy()
sub['pred'] = [label_cols[np.argmax(i/len(seeds))] for i in class_probs]

In [31]:
test_data['Category'].value_counts()

Corporate Information         26
AR Report / Market Insight    11
Offering                      10
Business Partnership           6
Award/Contest Winner           4
Merger & Acquisition           4
Others                         3
Won Project                    1
Name: Category, dtype: int64

In [32]:
sub['pred'].value_counts()

Corporate Information         51
Others                         5
AR Report / Market Insight     4
Offering                       3
Merger & Acquisition           2
Name: pred, dtype: int64

In [33]:
#5
from sklearn.metrics import classification_report
print(classification_report(test_data['Category'],sub['pred']))

                            precision    recall  f1-score   support

AR Report / Market Insight       0.75      0.27      0.40        11
      Award/Contest Winner       0.00      0.00      0.00         4
      Business Partnership       0.00      0.00      0.00         6
     Corporate Information       0.41      0.81      0.55        26
      Merger & Acquisition       1.00      0.50      0.67         4
                  Offering       0.67      0.20      0.31        10
                    Others       0.20      0.33      0.25         3
               Won Project       0.00      0.00      0.00         1

                  accuracy                           0.45        65
                 macro avg       0.38      0.26      0.27        65
              weighted avg       0.46      0.45      0.39        65



/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
final = sub[['url','text','pred']].copy()
final['actual'] = test_data['Category']
final.head()

url  \
ID                                                      
0   https://us.nttdata.com/en/news/press-release/2...   
1   https://www.nttdata.com/global/en/media/press-...   
2   https://www.nttdata.com/global/en/media/press-...   
3   https://www.nttdata.com/global/en/media/press-...   
4   https://uk.nttdata.com/news/ntt-data-uk-contin...   

                                                 text                   pred  \
ID                                                                             
0   organizations technology group group indycar s...  Corporate Information   
1   announcement 12 corporation corporation announ...  Corporate Information   
2   paper architectural global corporation global ...  Corporate Information   
3   services 15 corporation 15 global services ann...  Corporate Information   
4   08 announced graduation tech microsoft technol...  Corporate Information   

                   actual  
ID                         
0   Corporate Information  
1   Corporate Information  
2   Corporate Information  
3   Corporate Information  
4   Corporate Information

In [35]:
final['pred'].value_counts()

Corporate Information         51
Others                         5
AR Report / Market Insight     4
Offering                       3
Merger & Acquisition           2
Name: pred, dtype: int64

In [36]:
final.to_csv(r"/mnt/batch/tasks/shared/LS_root/mounts/clusters/dsvdm/code/Users/Tamilselvan.S/Press Release/scarp_5epoch_20sep.csv",
             index=False)

In [37]:
import pandas as pd
import re

final = pd.read_csv(r"/mnt/batch/tasks/shared/LS_root/mounts/clusters/dsvdm/code/Users/Tamilselvan.S/Press Release/scarp_5epoch_20sep.csv")
final.shape

(65, 4)

In [38]:
# def survey(x):
#     res = re.findall("survey",str(x).lower())
#     if len(res)>0:
#         return 'AR Report / Market Insight'
    

# def award(x):
#     res = re.findall("award",str(x).lower())
#     if len(res)>0:
#         return 'Award/Contest Winner'
    
# def acquire(x):
#     res = re.findall("acquire",str(x).lower())
#     if len(res)>0:
#         return 'Merger & Acquisition'
    
def overall(x):
    res1 = re.findall("survey",str(x).lower())
    res2 = re.findall("award",str(x).lower())
    res3 = re.findall("acquire",str(x).lower())
    if "survey" in res1:
        return 'AR Report / Market Insight'
    if "award" in res2:
        return 'Award/Contest Winner'
    if "acquire" in res3:
        return 'Merger & Acquisition'
    else:
        return 'null'

final['pred1'] = 'null'
final['pred1'] = final['text'].apply(lambda x: overall(x))

In [39]:
final['pred1'].value_counts()

null                    61
Merger & Acquisition     4
Name: pred1, dtype: int64

In [40]:
final.loc[final['pred1']=='null','pred1'] = final[final['pred1']=='null']['pred']

In [41]:
print(classification_report(final['actual'],final['pred']))

                            precision    recall  f1-score   support

AR Report / Market Insight       0.75      0.27      0.40        11
      Award/Contest Winner       0.00      0.00      0.00         4
      Business Partnership       0.00      0.00      0.00         6
     Corporate Information       0.41      0.81      0.55        26
      Merger & Acquisition       1.00      0.50      0.67         4
                  Offering       0.67      0.20      0.31        10
                    Others       0.20      0.33      0.25         3
               Won Project       0.00      0.00      0.00         1

                  accuracy                           0.45        65
                 macro avg       0.38      0.26      0.27        65
              weighted avg       0.46      0.45      0.39        65



/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
print(classification_report(final['actual'],final['pred1']))

                            precision    recall  f1-score   support

AR Report / Market Insight       0.75      0.27      0.40        11
      Award/Contest Winner       0.00      0.00      0.00         4
      Business Partnership       0.00      0.00      0.00         6
     Corporate Information       0.41      0.77      0.53        26
      Merger & Acquisition       0.50      0.50      0.50         4
                  Offering       0.67      0.20      0.31        10
                    Others       0.20      0.33      0.25         3
               Won Project       0.00      0.00      0.00         1

                  accuracy                           0.43        65
                 macro avg       0.32      0.26      0.25        65
              weighted avg       0.43      0.43      0.37        65



/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
final.to_csv(r"/mnt/batch/tasks/shared/LS_root/mounts/clusters/dsvdm/code/Users/Tamilselvan.S/Press Release/scarp_5epoch_20sep.csv",
             index=False)

In [194]:
#8
print(classification_report(test_data['Category'],sub['pred']))

                            precision    recall  f1-score   support

AR Report / Market Insight       0.68      0.60      0.64        25
      Award/Contest Winner       1.00      0.20      0.33        10
      Business Partnership       0.31      0.67      0.42         6
     Corporate Information       0.53      0.75      0.62        32
      Merger & Acquisition       1.00      0.33      0.50         3
                  Offering       0.60      0.46      0.52        13
                    Others       0.67      0.55      0.60        11
               Won Project       0.00      0.00      0.00         2

                  accuracy                           0.57       102
                 macro avg       0.60      0.44      0.45       102
              weighted avg       0.63      0.57      0.56       102



/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [181]:
from sklearn.metrics import classification_report
#5
print(classification_report(test_data['Category'],sub['pred']))

                            precision    recall  f1-score   support

AR Report / Market Insight       0.73      0.44      0.55        25
      Award/Contest Winner       1.00      0.20      0.33        10
      Business Partnership       0.27      0.50      0.35         6
     Corporate Information       0.60      0.84      0.70        32
      Merger & Acquisition       1.00      0.33      0.50         3
                  Offering       0.47      0.69      0.56        13
                    Others       0.89      0.73      0.80        11
               Won Project       0.00      0.00      0.00         2

                  accuracy                           0.60       102
                 macro avg       0.62      0.47      0.48       102
              weighted avg       0.67      0.60      0.58       102



/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [170]:
# import gym
# import numpy as np
# import time

# def init_q(s, a, type="ones"):
#     if type == "ones":
#         return np.ones((s, a))
#     elif type == "random":
#         return np.random.random((s, a))
#     elif type == "zeros":
#         return np.zeros((s, a))


# def epsilon_greedy(Q, epsilon, n_actions, s, train=False):
#     if train or np.random.rand() < epsilon:
#         action = np.argmax(Q[s, :])
#     else:
#         action = np.random.randint(0, n_actions)
#     return action

# def sarsa(alpha, gamma, epsilon, episodes, max_steps, n_tests, render = False, test=False):
#     env = gym.make('Taxi-v3')
#     n_states, n_actions = env.observation_space.n, env.action_space.n
#     Q = init_q(n_states, n_actions, type="ones")
#     timestep_reward = []
#     for episode in range(episodes):
#         total_reward = 0
#         s = env.reset()
#         a = epsilon_greedy(Q, epsilon, n_actions, s)
#         t = 0
#         done = False
#         while t < max_steps:
#             if render:
#                 env.render()
#             t += 1
#             s_, reward, done, info = env.step(a)
#             total_reward += reward
#             a_ = epsilon_greedy(Q, epsilon, n_actions, s_)
#             if done:
#                 Q[s, a] += alpha * ( reward  - Q[s, a] )
#             else:
#                 Q[s, a] += alpha * ( reward + (gamma * Q[s_, a_] ) - Q[s, a] )
#             s, a = s_, a_
#             if done:
#                 if render:
#                     print(f"This episode took {t} timesteps and reward {total_reward}")
#                 timestep_reward.append(total_reward)
#                 break
#     if render:
#         print(f"Here are the Q values:\n{Q}\nTesting now:")
#     if test:
#         test_agent(Q, env, n_tests, n_actions)
#     return timestep_reward

# def test_agent(Q, env, n_tests, n_actions, delay=0.1):
#     for test in range(episodes):
#         print(f"Test #{test}")
#         s = env.reset()
#         done = False
#         epsilon = 0
#         total_reward = 0
#         while not done:
#             time.sleep(delay)
#             env.render()
#             a = epsilon_greedy(Q, epsilon, n_actions, s, train=True)
#             print(f"Chose action {a} for state {s}")
#             s, reward, done, info = env.step(a)
#         total_reward += reward
#         if done:
#             print(f"Episode reward: {total_reward}")
#             time.sleep(1)
#             break

# alpha = 0.7
# gamma = 0.6
# epsilon = 0.4
# episodes = 1000
# max_steps = 50
# n_tests = 25
# timestep_reward = sarsa(alpha, gamma, epsilon, episodes, max_steps, n_tests)
# #     print(timestep_reward)